In [3]:
import findspark
findspark.init()

In [4]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("MovieSimilaritiesRedux002")
sc = SparkContext(conf = conf)

In [5]:
def process_movie_row(movie_row):
    movie=movie_row.split()
    user_id=int(movie[0])
    movie_id=int(movie[1])
    rating=float(movie[2])
    return user_id,(movie_id,rating)

user_movie_ratings=(sc.textFile("/home/ggomarr/Documents/Education/Udemy_Spark/ml-100k/u.data")
                      .map(process_movie_row))

In [35]:
for r in user_movie_ratings.takeOrdered(20):
    print(r)

(1, (1, 5.0))
(1, (2, 3.0))
(1, (3, 4.0))
(1, (4, 3.0))
(1, (5, 3.0))
(1, (6, 5.0))
(1, (7, 4.0))
(1, (8, 1.0))
(1, (9, 5.0))
(1, (10, 3.0))
(1, (11, 2.0))
(1, (12, 5.0))
(1, (13, 5.0))
(1, (14, 5.0))
(1, (15, 5.0))
(1, (16, 5.0))
(1, (17, 3.0))
(1, (18, 4.0))
(1, (19, 5.0))
(1, (20, 4.0))


In [ ]:
for r in user_movie_ratings.takeOrdered(10,key=lambda x: x[1][1]):
    print(r)

In [10]:
movie_avg_ratings=(user_movie_ratings.map(lambda x: (x[1][0],(x[1][1],1)))
                                     .reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1]))
                                     .mapValues(lambda x: (x[0]/x[1],x[1])))

In [11]:
for r in movie_avg_ratings.takeOrdered(10,key=lambda x: (x[1][0],-x[1][1])):
    print(r)

(314, (1.0, 5))
(437, (1.0, 5))
(439, (1.0, 5))
(858, (1.0, 3))
(1408, (1.0, 3))
(1432, (1.0, 3))
(784, (1.0, 2))
(1308, (1.0, 2))
(1334, (1.0, 2))
(1354, (1.0, 2))


In [54]:
min_rating=4

movie_avg_ratings_filtered=movie_avg_ratings.filter(lambda x: x[1][0]>=min_rating)

In [55]:
for r in movie_avg_ratings_filtered.takeOrdered(10,key=lambda x: (x[1][0],-x[1][1])):
    print(r)

(430, (4.0, 93))
(505, (4.0, 68))
(512, (4.0, 57))
(494, (4.0, 56))
(656, (4.0, 44))
(57, (4.0, 40))
(601, (4.0, 20))
(1143, (4.0, 15))
(1131, (4.0, 13))
(701, (4.0, 10))


In [56]:
user_movie_ratings_filtered=(user_movie_ratings.map(lambda x: (x[1][0],(x[0],x[1][1])))
                                               .join(movie_avg_ratings_filtered)
                                               .map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1][0]))))

In [57]:
for r in user_movie_ratings_filtered.takeOrdered(20):
    print(r)

(1, (12, 5.0, 4.385767790262173))
(1, (22, 4.0, 4.151515151515151))
(1, (23, 4.0, 4.1208791208791204))
(1, (45, 5.0, 4.05))
(1, (48, 5.0, 4.094017094017094))
(1, (50, 5.0, 4.3584905660377355))
(1, (56, 4.0, 4.060913705583756))
(1, (57, 5.0, 4.0))
(1, (59, 5.0, 4.0602409638554215))
(1, (60, 5.0, 4.015625))
(1, (64, 5.0, 4.445229681978798))
(1, (79, 4.0, 4.044642857142857))
(1, (83, 3.0, 4.0625))
(1, (89, 5.0, 4.138181818181818))
(1, (96, 5.0, 4.0067796610169495))
(1, (98, 4.0, 4.28974358974359))
(1, (100, 5.0, 4.155511811023622))
(1, (113, 5.0, 4.111111111111111))
(1, (114, 5.0, 4.447761194029851))
(1, (119, 5.0, 4.5))


In [58]:
user_mr_pairs=(user_movie_ratings_filtered.join(user_movie_ratings_filtered)
                                          .filter(lambda x: x[1][0][0]<x[1][1][0])
                                          .map(lambda x: ((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
                                          .groupByKey())

In [59]:
for r in user_mr_pairs.take(10):
    print(r)

((50, 96), <pyspark.resultiterable.ResultIterable object at 0x7fc939207a10>)
((223, 651), <pyspark.resultiterable.ResultIterable object at 0x7fc939207a90>)
((275, 487), <pyspark.resultiterable.ResultIterable object at 0x7fc939207c10>)
((100, 1498), <pyspark.resultiterable.ResultIterable object at 0x7fc939207450>)
((59, 511), <pyspark.resultiterable.ResultIterable object at 0x7fc939207b10>)
((223, 1131), <pyspark.resultiterable.ResultIterable object at 0x7fc939207e10>)
((60, 98), <pyspark.resultiterable.ResultIterable object at 0x7fc939207590>)
((60, 178), <pyspark.resultiterable.ResultIterable object at 0x7fc939207b50>)
((22, 484), <pyspark.resultiterable.ResultIterable object at 0x7fc939207550>)
((223, 867), <pyspark.resultiterable.ResultIterable object at 0x7fc939207410>)


In [60]:
def cosine_similarity(ratings):
    from math import sqrt
    def mult_vectors(a,b):
        return sum([1.0*a[n]*b[n] for n in range(len(a))])
    v1,v2=zip(*ratings)
    num_pairs=len(v1)
    try:
        return mult_vectors(v1,v2)/(sqrt(mult_vectors(v1,v1)) * sqrt(mult_vectors(v2,v2))),num_pairs
    except:
        return 0,num_pairs
    
movie_pair_similarities=user_mr_pairs.mapValues(cosine_similarity)

In [61]:
for r in movie_pair_similarities.take(10):
    print(r)

((50, 96), (0.9638252985093606, 271))
((223, 651), (0.9704414075796239, 55))
((275, 487), (0.9760904982155516, 43))
((100, 1498), (1.0, 1))
((59, 511), (0.9663140130917418, 37))
((223, 1131), (0.9570202978345285, 6))
((60, 98), (0.9517171195132221, 48))
((60, 178), (0.9619532111324821, 26))
((22, 484), (0.9623569271269912, 70))
((223, 867), (0.9775339768579192, 4))


In [62]:
def process_movie_row(movie_row):
    movie=movie_row.split('|')
    movie_id=int(movie[0])
    movie_title=movie[1]
    return (movie_id,movie_title)

movie_names_and_ratings=(sc.textFile("/home/ggomarr/Documents/Education/Udemy_Spark/ml-100k/u.item")
                           .map(process_movie_row)
                           .join(movie_avg_ratings)
                           .mapValues(lambda x: (x[0],x[1][0],x[1][1])))

In [63]:
for r in movie_names_and_ratings.takeOrdered(10):
    print('{:4d} [{:4.2f}, {:3d}] - {}'.format(r[0],r[1][1],r[1][2],r[1][0]))

   1 [3.88, 452] - Toy Story (1995)
   2 [3.21, 131] - GoldenEye (1995)
   3 [3.03,  90] - Four Rooms (1995)
   4 [3.55, 209] - Get Shorty (1995)
   5 [3.30,  86] - Copycat (1995)
   6 [3.58,  26] - Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
   7 [3.80, 392] - Twelve Monkeys (1995)
   8 [4.00, 219] - Babe (1995)
   9 [3.90, 299] - Dead Man Walking (1995)
  10 [3.83,  89] - Richard III (1995)


In [64]:
target_movie=50
min_similarity=0.95
min_num_rating_pairs=25

target_movie_filtered=(movie_pair_similarities.filter(lambda x: ((x[0][0]==target_movie) or \
                                                                 (x[0][1]==target_movie)) and \
                                                                 x[1][0]>=min_similarity and \
                                                                 x[1][1]>=min_num_rating_pairs)
                                              .map(lambda x: (x[0][0] if x[0][0]<>target_movie else x[0][1],
                                                              (x[1][0],x[1][1]))))

In [65]:
for r in target_movie_filtered.takeOrdered(10,key=lambda x: (-x[1][0],-x[1][1])):
    print(r)

(172, (0.9895522078385338, 345))
(181, (0.9857230861253026, 480))
(963, (0.9823449614960231, 40))
(174, (0.981760098872619, 380))
(1007, (0.9783184758610347, 37))
(178, (0.9776576120448436, 109))
(408, (0.9775948291054827, 92))
(498, (0.9764692222674887, 138))
(251, (0.9761507096081464, 38))
(1142, (0.9758709250445461, 37))


In [66]:
target_movie_filtered_named_rated=(target_movie_filtered.join(movie_names_and_ratings)
                                                        .mapValues(lambda x: (x[1][0],x[1][1],x[1][2],
                                                                              x[0][1],x[0][0])))

In [67]:
for r in target_movie_filtered_named_rated.take(10):
    print r

(12, (u'Usual Suspects, The (1995)', 4.385767790262173, 267, 223, 0.9724956031333988))
(528, (u'Killing Fields, The (1984)', 4.132231404958677, 121, 101, 0.9606248098770778))
(48, (u'Hoop Dreams (1994)', 4.094017094017094, 117, 104, 0.9596593446508168))
(96, (u'Terminator 2: Judgment Day (1991)', 4.0067796610169495, 295, 271, 0.9638252985093606))
(132, (u'Wizard of Oz, The (1939)', 4.0772357723577235, 246, 221, 0.9686687782994247))
(648, (u'Quiet Man, The (1952)', 4.029850746268656, 67, 57, 0.9602369649769346))
(168, (u'Monty Python and the Holy Grail (1974)', 4.0664556962025316, 316, 278, 0.9628570039980153))
(180, (u'Apocalypse Now (1979)', 4.04524886877828, 221, 183, 0.9554367900581618))
(408, (u'Close Shave, A (1995)', 4.491071428571429, 112, 92, 0.9775948291054827))
(480, (u'North by Northwest (1959)', 4.284916201117318, 179, 156, 0.9734534315266805))


In [68]:
target_movie_r=movie_names_and_ratings.lookup(target_movie)[0]
print('Most similar movies to {:4d} [{:4.2f}, {:3d}] - {}'.format(target_movie,
                                                                  target_movie_r[1],target_movie_r[2],
                                                                  target_movie_r[0]))
for r in target_movie_filtered_named_rated.takeOrdered(10,key=lambda x: (-x[1][4],-x[1][1])):
    print('{:4d} [{:.2f}, {:3d}/{:3d}] ({:.4f}): {}'.format(r[0],r[1][1],r[1][3],r[1][2],r[1][4],r[1][0]))

Most similar movies to   50 [4.36, 583] - Star Wars (1977)
 172 [4.20, 345/367] (0.9896): Empire Strikes Back, The (1980)
 181 [4.01, 480/507] (0.9857): Return of the Jedi (1983)
 963 [4.29,  40/ 41] (0.9823): Some Folks Call It a Sling Blade (1993)
 174 [4.25, 380/420] (0.9818): Raiders of the Lost Ark (1981)
1007 [4.13,  37/ 47] (0.9783): Waiting for Guffman (1996)
 178 [4.34, 109/125] (0.9777): 12 Angry Men (1957)
 408 [4.49,  92/112] (0.9776): Close Shave, A (1995)
 498 [4.18, 138/152] (0.9765): African Queen, The (1951)
 251 [4.26,  38/ 46] (0.9762): Shall We Dance? (1996)
1142 [4.05,  37/ 44] (0.9759): When We Were Kings (1996)
